# PRMT-2424 Integration times for a practice - July - September 2021

## Context
We wanted to verify integration times for a practice

## Scope

Produce a breakdown of integration times

Check the data in the dashboard matches what we expect from the raw data.

Get timestamps and sending practice names for the records that were not integrated within 3 days.

For the records not integrated in 8 days - are they now integrated? How long did it take for them to be integrated?

In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

In [7]:
asid_lookup=read_asid_metadata("prm-gp2gp-ods-metadata-prod", "v2/2021/10/organisationMetadata.json")

transfer_file_location = "s3://prm-gp2gp-transfer-data-prod/v5/2021/"

transfer_files = [
    "9/2021-9-transfers.parquet",
    "7/2021-7-transfers.parquet",
    "8/2021-8-transfers.parquet"
]
transfer_input_files = [transfer_file_location + f for f in transfer_files]

transfers_raw = pd.concat((
    pd.read_parquet(f)
    for f in transfer_input_files
))

transfers = transfers_raw\
    .join(asid_lookup.add_prefix("requesting_"), on="requesting_practice_asid", how="left")\
    .join(asid_lookup.add_prefix("sending_"), on="sending_practice_asid", how="left")\

transfers['month']=transfers['date_requested'].dt.to_period('M')

# Supplier name mapping
supplier_renaming = {
    "EGTON MEDICAL INFORMATION SYSTEMS LTD (EMIS)":"EMIS",
    "IN PRACTICE SYSTEMS LTD":"Vision",
    "MICROTEST LTD":"Microtest",
    "THE PHOENIX PARTNERSHIP":"TPP",
    "SystmOne" : "TPP",
    None: "Unknown"
}

transfers["sending_supplier"] = transfers["sending_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())
transfers["requesting_supplier"] = transfers["requesting_supplier"].replace(supplier_renaming.keys(), supplier_renaming.values())

In [3]:
# show all columns
pd.set_option('display.max_columns', None)

# filter for the practice as the requester with process failure
practice_of_interest_bool = transfers["requesting_practice_ods_code"]==""
process_failure_bool = transfers["status"] == "Process failure"
practice_of_interest_transfers = transfers[practice_of_interest_bool & process_failure_bool].copy()

practice_of_interest_transfers

,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,...,date_completed,requesting_practice_ods_code,requesting_practice_name,requesting_ccg_ods_code,requesting_ccg_name,sending_practice_ods_code,sending_practice_name,sending_ccg_ods_code,sending_ccg_name,month
62589,55085385-D26C-4637-A82F-0860E9AA0EED,1204722.0,512070360015,200000010476,EMIS,EMIS,[nan],[nan],[],Process failure,...,2021-09-17 06:13:14.387,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,Y03079,BOLTON COMMUNITY PRACTICE,00T,NHS BOLTON CCG,2021-09
166736,C868C9E8-9D24-40B8-8A41-85CE944490AA,NaN,512070360015,694466006038,EMIS,Vision,[nan],[],[],Process failure,...,NaT,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,P83009,BLACKFORD HOUSE MEDICAL CENTRE,00V,NHS BURY CCG,2021-09
1098,1748B8E9-B439-4EFB-83C4-1907221F9CC4,744295.0,512070360015,200000024011,EMIS,EMIS,[nan],[nan],[],Process failure,...,2021-07-23 06:11:32.278,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,P87035,ORDSALL HEALTH SURGERY,01G,NHS SALFORD CCG,2021-07
12545,749C08CF-6663-48D3-8016-256D85775ABD,775270.0,512070360015,506071244012,EMIS,EMIS,[nan],[nan],[],Process failure,...,2021-07-23 06:09:59.183,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,P82031,HEATON MEDICAL CENTRE,00T,NHS BOLTON CCG,2021-07
96607,A717197F-BF91-4CFB-8CD0-47B0275949C6,NaN,512070360015,200000010476,EMIS,EMIS,[nan],[],[],Process failure,...,NaT,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,Y03079,BOLTON COMMUNITY PRACTICE,00T,NHS BOLTON CCG,2021-08
116763,A6CF344B-429B-44A8-A69E-150C99F23093,NaN,512070360015,200000015073,EMIS,EMIS,[nan],[],[],Process failure,...,NaT,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,P82012,DR EARNSHAW AND PARTNERS,00T,NHS BOLTON CCG,2021-08


In [4]:
# filter for the practice as the requester with status - Integrated on time
integrated_on_time_bool = transfers["status"]=="Integrated on time"
transfers_integrated_on_time_beyond_3_days = (transfers["date_completed"] - timedelta(days=3)) > transfers["date_requested"] 

practice_of_interest_transfers_integrated_on_time_beyond_3_days = transfers[practice_of_interest_bool & integrated_on_time_bool & transfers_integrated_on_time_beyond_3_days].copy()

practice_of_interest_transfers_integrated_on_time_beyond_3_days


,conversation_id,sla_duration,requesting_practice_asid,sending_practice_asid,requesting_supplier,sending_supplier,sender_error_codes,final_error_codes,intermediate_error_codes,status,...,date_completed,requesting_practice_ods_code,requesting_practice_name,requesting_ccg_ods_code,requesting_ccg_name,sending_practice_ods_code,sending_practice_name,sending_ccg_ods_code,sending_ccg_name,month
129739,DE8FF843-854B-4465-9D82-2508BE9C4F0D,283108.0,512070360015,200000014663,EMIS,TPP,[nan],[nan],[],Integrated on time,...,2021-09-24 19:07:08.016,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,P92016,PENNYGATE MEDICAL CENTRE,02H,NHS WIGAN BOROUGH CCG,2021-09
109215,03BC74A2-FBBC-41E9-8893-2E1E294C29B5,293709.0,512070360015,293701819048,EMIS,EMIS,[nan],[nan],[],Integrated on time,...,2021-07-31 19:51:07.584,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,A86003,SAVILLE MEDICAL GROUP,13T,NHS NEWCASTLE GATESHEAD CCG,2021-07
116788,208A74EA-D2E8-40DF-AFDE-B9B9E9483B5B,161312.0,512070360015,876611333017,EMIS,EMIS,"[nan, nan]",[nan],[],Integrated on time,...,2021-09-03 07:35:29.682,P82001,THE DUNSTAN PARTNERSHIP,00T,NHS BOLTON CCG,N81072,MURDISHAW,01F,NHS HALTON CCG,2021-08
